<a href="https://colab.research.google.com/github/yebyyy/Tokenization/blob/main/lecture.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### This notebook is for tokenization

Tokens are atoms of a llm

#### Byte Level Tokenization
Tiktokenizer.vercel.app -> GPT 2

English: space with the next word, single word sometimes changes, case sensitive

Numbers: kinda arbitrary, sometimes single sometimes muiltiple

Non English language: slightly worse than English, longer tokens for English than in other languages, so same thing will lead to more tokens in other languages, chunks broken up more, thus same maximum context window will lead to worse performance

Code: leading space handled individually, wasting tokens

GPT4 -> Token numbers dropped, python improved

Not less better, there is an appropriate number

----------------------------------------------------------------

#### Why not just use the Unicode?
- Unicode changes every once in a while

#### UTF-8
Based on the Unicode, turn characters into 1~4 byte representations

In [2]:
list("你好我是Robbie".encode("utf-8"))

[228,
 189,
 160,
 229,
 165,
 189,
 230,
 136,
 145,
 230,
 152,
 175,
 82,
 111,
 98,
 98,
 105,
 101]

In [3]:
list("你好我是Robbie".encode("utf-16"))

[255,
 254,
 96,
 79,
 125,
 89,
 17,
 98,
 47,
 102,
 82,
 0,
 111,
 0,
 98,
 0,
 98,
 0,
 105,
 0,
 101,
 0]

In [4]:
list("你好我是Robbie".encode("utf-16"))

[255,
 254,
 96,
 79,
 125,
 89,
 17,
 98,
 47,
 102,
 82,
 0,
 111,
 0,
 98,
 0,
 98,
 0,
 105,
 0,
 101,
 0]

The thing with UTF-16 and UTF-32 is the 0s here representing the waste of tokens

#### Why not just use UTF-8?
- Since 1 byte is only a size of 256, which is very limited, and the text will be very stretched out, as demonstrated in the UTF-8 encode list

#### Solution: Byte Pair Encoding (BPE) Algorithm

- Iteratively replace the most common contiguous sequences of characters in our vocabulary, and replace every occurrence of it by a new token(not in the vocabulary)

### Encoding

In [5]:
text = "Ｕｎｉｃｏｄｅ! 🅤🅝🅘🅒🅞🅓🅔‽ 🇺‌🇳‌🇮‌🇨‌🇴‌🇩‌🇪! 😄 The very name strikes fear and awe into the hearts of programmers worldwide. We all know we ought to “support Unicode” in our software (whatever that means—like using wchar_t for all the strings, right?). But Unicode can be abstruse, and diving into the thousand-page Unicode Standard plus its dozens of supplementary annexes, reports, and notes can be more than a little intimidating. I don’t blame programmers for still finding the whole thing mysterious, even 30 years after Unicode’s inception."
utf8 = list(map(int, text.encode("utf-8")))
len(text) < len(utf8)

True

Some of the characters have more than one byte

Find the most common pair:

In [6]:
def find_most_common(intList):
  counts = {}
  for pair in zip(intList, intList[1:]):
    counts[pair] = counts.get(pair, 0) + 1
  return counts
common = find_most_common(utf8)
commonList = sorted([(c, p) for p, c in common.items()], reverse=True)
commonList

[(20, (101, 32)),
 (15, (240, 159)),
 (12, (226, 128)),
 (12, (105, 110)),
 (10, (115, 32)),
 (10, (97, 110)),
 (10, (32, 97)),
 (9, (32, 116)),
 (8, (116, 104)),
 (7, (159, 135)),
 (7, (159, 133)),
 (7, (97, 114)),
 (6, (239, 189)),
 (6, (140, 240)),
 (6, (128, 140)),
 (6, (116, 32)),
 (6, (114, 32)),
 (6, (111, 114)),
 (6, (110, 103)),
 (6, (110, 100)),
 (6, (109, 101)),
 (6, (104, 101)),
 (6, (101, 114)),
 (6, (32, 105)),
 (5, (117, 115)),
 (5, (115, 116)),
 (5, (110, 32)),
 (5, (100, 101)),
 (5, (44, 32)),
 (5, (32, 115)),
 (4, (116, 105)),
 (4, (116, 101)),
 (4, (115, 44)),
 (4, (114, 105)),
 (4, (111, 117)),
 (4, (111, 100)),
 (4, (110, 116)),
 (4, (110, 105)),
 (4, (105, 99)),
 (4, (104, 97)),
 (4, (103, 32)),
 (4, (101, 97)),
 (4, (100, 32)),
 (4, (99, 111)),
 (4, (97, 109)),
 (4, (85, 110)),
 (4, (32, 119)),
 (4, (32, 111)),
 (4, (32, 102)),
 (4, (32, 85)),
 (3, (118, 101)),
 (3, (116, 115)),
 (3, (116, 114)),
 (3, (116, 111)),
 (3, (114, 116)),
 (3, (114, 115)),
 (3, (114, 10

In [7]:
top_pair = max(common, key=common.get)
top_pair

(101, 32)

In [8]:
chr(101), chr(32)

('e', ' ')

'e ' is the most common pair in this iteration

- Now we need to iterate over the text and replace 'e ' with a character of ord(char) = 256, since the given text is from 0-255

In [9]:
def replace_key(intList, pair, replacement):
  newList = []
  i = 0
  while i < len(intList):
    if i < len(intList) - 1 and intList[i] == pair[0] and intList[i+1] == pair[1]:
      newList.append(replacement)
      i += 2
    else:
      newList.append(intList[i])
      i += 1
  return newList

tokens = replace_key(utf8, top_pair, 256)
tokens, len(tokens)

([239,
  188,
  181,
  239,
  189,
  142,
  239,
  189,
  137,
  239,
  189,
  131,
  239,
  189,
  143,
  239,
  189,
  132,
  239,
  189,
  133,
  33,
  32,
  240,
  159,
  133,
  164,
  240,
  159,
  133,
  157,
  240,
  159,
  133,
  152,
  240,
  159,
  133,
  146,
  240,
  159,
  133,
  158,
  240,
  159,
  133,
  147,
  240,
  159,
  133,
  148,
  226,
  128,
  189,
  32,
  240,
  159,
  135,
  186,
  226,
  128,
  140,
  240,
  159,
  135,
  179,
  226,
  128,
  140,
  240,
  159,
  135,
  174,
  226,
  128,
  140,
  240,
  159,
  135,
  168,
  226,
  128,
  140,
  240,
  159,
  135,
  180,
  226,
  128,
  140,
  240,
  159,
  135,
  169,
  226,
  128,
  140,
  240,
  159,
  135,
  170,
  33,
  32,
  240,
  159,
  152,
  132,
  32,
  84,
  104,
  256,
  118,
  101,
  114,
  121,
  32,
  110,
  97,
  109,
  256,
  115,
  116,
  114,
  105,
  107,
  101,
  115,
  32,
  102,
  101,
  97,
  114,
  32,
  97,
  110,
  100,
  32,
  97,
  119,
  256,
  105,
  110,
  116,
  111,
  32,
 

#### Training the tokenizer
Putting everything together

In [10]:
text = "A Programmer’s Introduction to UnicodeMarch 3, 2017 · Coding · 24 CommentsＵｎｉｃｏｄｅ! 🅤🅝🅘🅒🅞🅓🅔‽ 🇺‌🇳‌🇮‌🇨‌🇴‌🇩‌🇪! 😄 The very name strikes fear and awe into the hearts of programmers worldwide. We all know we ought to “support Unicode” in our software (whatever that means—like using wchar_t for all the strings, right?). But Unicode can be abstruse, and diving into the thousand-page Unicode Standard plus its dozens of supplementary annexes, reports, and notes can be more than a little intimidating. I don’t blame programmers for still finding the whole thing mysterious, even 30 years after Unicode’s inception.A few months ago, I got interested in Unicode and decided to spend some time learning more about it in detail. In this article, I’ll give an introduction to it from a programmer’s point of view.I’m going to focus on the character set and what’s involved in working with strings and files of Unicode text. However, in this article I’m not going to talk about fonts, text layout/shaping/rendering, or localization in detail—those are separate issues, beyond my scope (and knowledge) here.Diversity and Inherent ComplexityThe Unicode CodespaceCodespace AllocationScriptsUsage FrequencyEncodingsUTF-8UTF-16Combining MarksCanonical EquivalenceNormalization FormsGrapheme ClustersAnd More…Diversity and Inherent ComplexityAs soon as you start to study Unicode, it becomes clear that it represents a large jump in complexity over character sets like ASCII that you may be more familiar with. It’s not just that Unicode contains a much larger number of characters, although that’s part of it. Unicode also has a great deal of internal structure, features, and special cases, making it much more than what one might expect a mere “character set” to be. We’ll see some of that later in this article.When confronting all this complexity, especially as an engineer, it’s hard not to find oneself asking, “Why do we need all this? Is this really necessary? Couldn’t it be simplified?”However, Unicode aims to faithfully represent the entire world’s writing systems. The Unicode Consortium’s stated goal is “enabling people around the world to use computers in any language”. And as you might imagine, the diversity of written languages is immense! To date, Unicode supports 135 different scripts, covering some 1100 languages, and there’s still a long tail of over 100 unsupported scripts, both modern and historical, which people are still working to add.Given this enormous diversity, it’s inevitable that representing it is a complicated project. Unicode embraces that diversity, and accepts the complexity inherent in its mission to include all human writing systems. It doesn’t make a lot of trade-offs in the name of simplification, and it makes exceptions to its own rules where necessary to further its mission.Moreover, Unicode is committed not just to supporting texts in any single language, but also to letting multiple languages coexist within one text—which introduces even more complexity.Most programming languages have libraries available to handle the gory low-level details of text manipulation, but as a programmer, you’ll still need to know about certain Unicode features in order to know when and how to apply them. It may take some time to wrap your head around it all, but don’t be discouraged—think about the billions of people for whom your software will be more accessible through supporting text in their language. Embrace the complexity!The Unicode CodespaceLet’s start with some general orientation. The basic elements of Unicode—its “characters”, although that term isn’t quite right—are called code points. Code points are identified by number, customarily written in hexadecimal with the prefix “U+”, such as U+0041 “A” latin capital letter a or U+03B8 “θ” greek small letter theta. Each code point also has a short name, and quite a few other properties, specified in the Unicode Character Database.The set of all possible code points is called the codespace. The Unicode codespace consists of 1,114,112 code points. However, only 128,237 of them—about 12% of the codespace—are actually assigned, to date. There’s plenty of room for growth! Unicode also reserves an additional 137,468 code points as “private use” areas, which have no standardized meaning and are available for individual applications to define for their own purposes.Codespace AllocationTo get a feel for how the codespace is laid out, it’s helpful to visualize it. Below is a map of the entire codespace, with one pixel per code point. It’s arranged in tiles for visual coherence; each small square is 16×16 = 256 code points, and each large square is a “plane” of 65,536 code points. There are 17 planes altogether.Map of the Unicode codespace (click to zoom)White represents unassigned space. Blue is assigned code points, green is private-use areas, and the small red area is surrogates (more about those later). As you can see, the assigned code points are distributed somewhat sparsely, but concentrated in the first three planes.Plane 0 is also known as the “Basic Multilingual Plane”, or BMP. The BMP contains essentially all the characters needed for modern text in any script, including Latin, Cyrillic, Greek, Han (Chinese), Japanese, Korean, Arabic, Hebrew, Devanagari (Indian), and many more.(In the past, the codespace was just the BMP and no more—Unicode was originally conceived as a straightforward 16-bit encoding, with only 65,536 code points. It was expanded to its current size in 1996. However, the vast majority of code points in modern text belong to the BMP.)Plane 1 contains historical scripts, such as Sumerian cuneiform and Egyptian hieroglyphs, as well as emoji and various other symbols. Plane 2 contains a large block of less-common and historical Han characters. The remaining planes are empty, except for a small number of rarely-used formatting characters in Plane 14; planes 15–16 are reserved entirely for private use.ScriptsLet’s zoom in on the first three planes, since that’s where the action is:Map of scripts in Unicode planes 0–2 (click to zoom)This map color-codes the 135 different scripts in Unicode. You can see how Han () and Korean () take up most of the range of the BMP (the left large square). By contrast, all of the European, Middle Eastern, and South Asian scripts fit into the first row of the BMP in this diagram.Many areas of the codespace are adapted or copied from earlier encodings. For example, the first 128 code points of Unicode are just a copy of ASCII. This has clear benefits for compatibility—it’s easy to losslessly convert texts from smaller encodings into Unicode (and the other direction too, as long as no characters outside the smaller encoding are used).Usage FrequencyOne more interesting way to visualize the codespace is to look at the distribution of usage—in other words, how often each code point is actually used in real-world texts. Below is a heat map of planes 0–2 based on a large sample of text from Wikipedia and Twitter (all languages). Frequency increases from black (never seen) through red and yellow to white.Heat map of code point usage frequency in Unicode planes 0–2 (click to zoom)You can see that the vast majority of this text sample lies in the BMP, with only scattered usage of code points from planes 1–2. The biggest exception is emoji, which show up here as the several bright squares in the bottom row of plane 1.EncodingsWe’ve seen that Unicode code points are abstractly identified by their index in the codespace, ranging from U+0000 to U+10FFFF. But how do code points get represented as bytes, in memory or in a file?The most convenient, computer-friendliest (and programmer-friendliest) thing to do would be to just store the code point index as a 32-bit integer. This works, but it consumes 4 bytes per code point, which is sort of a lot. Using 32-bit ints for Unicode will cost you a bunch of extra storage, memory, and performance in bandwidth-bound scenarios, if you work with a lot of text.Consequently, there are several more-compact encodings for Unicode. The 32-bit integer encoding is officially called UTF-32 (UTF = “Unicode Transformation Format”), but it’s rarely used for storage. At most, it comes up sometimes as a temporary internal representation, for examining or operating on the code points in a string.Much more commonly, you’ll see Unicode text encoded as either UTF-8 or UTF-16. These are both variable-length encodings, made up of 8-bit or 16-bit units, respectively. In these schemes, code points with smaller index values take up fewer bytes, which saves a lot of memory for typical texts. The trade-off is that processing UTF-8/16 texts is more programmatically involved, and likely slower.UTF-8In UTF-8, each code point is stored using 1 to 4 bytes, based on its index value.UTF-8 uses a system of binary prefixes, in which the high bits of each byte mark whether it’s a single byte, the beginning of a multi-byte sequence, or a continuation byte; the remaining bits, concatenated, give the code point index. This table shows how it works:UTF-8 (binary)	Code point (binary)	Range0xxxxxxx	xxxxxxx	U+0000–U+007F110xxxxx 10yyyyyy	xxxxxyyyyyy	U+0080–U+07FF1110xxxx 10yyyyyy 10zzzzzz	xxxxyyyyyyzzzzzz	U+0800–U+FFFF11110xxx 10yyyyyy 10zzzzzz 10wwwwww	xxxyyyyyyzzzzzzwwwwww	U+10000–U+10FFFFA handy property of UTF-8 is that code points below 128 (ASCII characters) are encoded as single bytes, and all non-ASCII code points are encoded using sequences of bytes 128–255. This has a couple of nice consequences. First, any strings or files out there that are already in ASCII can also be interpreted as UTF-8 without any conversion. Second, lots of widely-used string programming idioms—such as null termination, or delimiters (newlines, tabs, commas, slashes, etc.)—will just work on UTF-8 strings. ASCII bytes never occur inside the encoding of non-ASCII code points, so searching byte-wise for a null terminator or a delimiter will do the right thing.Thanks to this convenience, it’s relatively simple to extend legacy ASCII programs and APIs to handle UTF-8 strings. UTF-8 is very widely used in the Unix/Linux and Web worlds, and many programmers argue UTF-8 should be the default encoding everywhere.However, UTF-8 isn’t a drop-in replacement for ASCII strings in all respects. For instance, code that iterates over the “characters” in a string will need to decode UTF-8 and iterate over code points (or maybe grapheme clusters—more about those later), not bytes. When you measure the “length” of a string, you’ll need to think about whether you want the length in bytes, the length in code points, the width of the text when rendered, or something else.UTF-16The other encoding that you’re likely to encounter is UTF-16. It uses 16-bit words, with each code point stored as either 1 or 2 words.Like UTF-8, we can express the UTF-16 encoding rules in the form of binary prefixes:UTF-16 (binary)	Code point (binary)	Rangexxxxxxxxxxxxxxxx	xxxxxxxxxxxxxxxx	U+0000–U+FFFF110110xxxxxxxxxx 110111yyyyyyyyyy	xxxxxxxxxxyyyyyyyyyy + 0x10000	U+10000–U+10FFFFA more common way that people talk about UTF-16 encoding, though, is in terms of code points called “surrogates”. All the code points in the range U+D800–U+DFFF—or in other words, the code points that match the binary prefixes 110110 and 110111 in the table above—are reserved specifically for UTF-16 encoding, and don’t represent any valid characters on their own. They’re only meant to occur in the 2-word encoding pattern above, which is called a “surrogate pair”. Surrogate code points are illegal in any other context! They’re not allowed in UTF-8 or UTF-32 at all.Historically, UTF-16 is a descendant of the original, pre-1996 versions of Unicode, in which there were only 65,536 code points. The original intention was that there would be no different “encodings”; Unicode was supposed to be a straightforward 16-bit character set. Later, the codespace was expanded to make room for a long tail of less-common (but still important) Han characters, which the Unicode designers didn’t originally plan for. Surrogates were then introduced, as—to put it bluntly—a kludge, allowing 16-bit encodings to access the new code points.Today, Javascript uses UTF-16 as its standard string representation: if you ask for the length of a string, or iterate over it, etc., the result will be in UTF-16 words, with any code points outside the BMP expressed as surrogate pairs. UTF-16 is also used by the Microsoft Win32 APIs; though Win32 supports either 8-bit or 16-bit strings, the 8-bit version unaccountably still doesn’t support UTF-8—only legacy code-page encodings, like ANSI. This leaves UTF-16 as the only way to get proper Unicode support in Windows. (Update: in Win10 version 1903, they finally added UTF-8 support to the 8-bit APIs! 😊)By the way, UTF-16’s words can be stored either little-endian or big-endian. Unicode has no opinion on that issue, though it does encourage the convention of putting U+FEFF zero width no-break space at the top of a UTF-16 file as a byte-order mark, to disambiguate the endianness. (If the file doesn’t match the system’s endianness, the BOM will be decoded as U+FFFE, which isn’t a valid code point.)Combining MarksIn the story so far, we’ve been focusing on code points. But in Unicode, a “character” can be more complicated than just an individual code point!Unicode includes a system for dynamically composing characters, by combining multiple code points together. This is used in various ways to gain flexibility without causing a huge combinatorial explosion in the number of code points.In European languages, for example, this shows up in the application of diacritics to letters. Unicode supports a wide range of diacritics, including acute and grave accents, umlauts, cedillas, and many more. All these diacritics can be applied to any letter of any alphabet—and in fact, multiple diacritics can be used on a single letter.If Unicode tried to assign a distinct code point to every possible combination of letter and diacritics, things would rapidly get out of hand. Instead, the dynamic composition system enables you to construct the character you want, by starting with a base code point (the letter) and appending additional code points, called “combining marks”, to specify the diacritics. When a text renderer sees a sequence like this in a string, it automatically stacks the diacritics over or under the base letter to create a composed character.For example, the accented character “Á” can be expressed as a string of two code points: U+0041 “A” latin capital letter a plus U+0301 “◌́” combining acute accent. This string automatically gets rendered as a single character: “Á”.Now, Unicode does also include many “precomposed” code points, each representing a letter with some combination of diacritics already applied, such as U+00C1 “Á” latin capital letter a with acute or U+1EC7 “ệ” latin small letter e with circumflex and dot below. I suspect these are mostly inherited from older encodings that were assimilated into Unicode, and kept around for compatibility. In practice, there are precomposed code points for most of the common letter-with-diacritic combinations in European-script languages, so they don’t use dynamic composition that much in typical text.Still, the system of combining marks does allow for an arbitrary number of diacritics to be stacked on any base character. The reductio-ad-absurdum of this is Zalgo text, which works by ͖͟ͅr͞aṋ̫̠̖͈̗d͖̻̹óm̪͙͕̗̝ļ͇̰͓̳̫ý͓̥̟͍ ̕s̫t̫̱͕̗̰̼̘͜a̼̩͖͇̠͈̣͝c̙͍k̖̱̹͍͘i̢n̨̺̝͇͇̟͙ģ̫̮͎̻̟ͅ ̕n̼̺͈͞u̮͙m̺̭̟̗͞e̞͓̰̤͓̫r̵o̖ṷs҉̪͍̭̬̝̤ ̮͉̝̞̗̟͠d̴̟̜̱͕͚i͇̫̼̯̭̜͡ḁ͙̻̼c̲̲̹r̨̠̹̣̰̦i̱t̤̻̤͍͙̘̕i̵̜̭̤̱͎c̵s ͘o̱̲͈̙͖͇̲͢n͘ ̜͈e̬̲̠̩ac͕̺̠͉h̷̪ ̺̣͖̱ḻ̫̬̝̹ḙ̙̺͙̭͓̲t̞̞͇̲͉͍t̷͔̪͉̲̻̠͙e̦̻͈͉͇r͇̭̭̬͖,̖́ ̜͙͓̣̭s̘̘͈o̱̰̤̲ͅ ̛̬̜̙t̼̦͕̱̹͕̥h̳̲͈͝ͅa̦t̻̲ ̻̟̭̦̖t̛̰̩h̠͕̳̝̫͕e͈̤̘͖̞͘y҉̝͙ ̷͉͔̰̠o̞̰v͈͈̳̘͜er̶f̰͈͔ḻ͕̘̫̺̲o̲̭͙͠ͅw̱̳̺ ͜t̸h͇̭͕̳͍e̖̯̟̠ ͍̞̜͔̩̪͜ļ͎̪̲͚i̝̲̹̙̩̹n̨̦̩̖ḙ̼̲̼͢ͅ ̬͝s̼͚̘̞͝p͙̘̻a̙c҉͉̜̤͈̯̖i̥͡n̦̠̱͟g̸̗̻̦̭̮̟ͅ ̳̪̠͖̳̯̕a̫͜n͝d͡ ̣̦̙ͅc̪̗r̴͙̮̦̹̳e͇͚̞͔̹̫͟a̙̺̙ț͔͎̘̹ͅe̥̩͍ a͖̪̜̮͙̹n̢͉̝ ͇͉͓̦̼́a̳͖̪̤̱p̖͔͔̟͇͎͠p̱͍̺ę̲͎͈̰̲̤̫a̯͜r̨̮̫̣̘a̩̯͖n̹̦̰͎̣̞̞c̨̦̱͔͎͍͖e̬͓͘ ̤̰̩͙̤̬͙o̵̼̻̬̻͇̮̪f̴ ̡̙̭͓͖̪̤“̸͙̠̼c̳̗͜o͏̼͙͔̮r̞̫̺̞̥̬ru̺̻̯͉̭̻̯p̰̥͓̣̫̙̤͢t̳͍̳̖ͅi̶͈̝͙̼̙̹o̡͔n̙̺̹̖̩͝ͅ”̨̗͖͚̩.̯͓A few other places where dynamic character composition shows up in Unicode:Vowel-pointing notation in Arabic and Hebrew. In these languages, words are normally spelled with some of their vowels left out. They then have diacritic notation to indicate the vowels (used in dictionaries, language-teaching materials, children’s books, and such). These diacritics are expressed with combining marks.A Hebrew example, with niqqud:	אֶת דַלְתִּי הֵזִיז הֵנִיעַ, קֶטֶב לִשְׁכַּתִּי יָשׁוֹדNormal writing (no niqqud):	את דלתי הזיז הניע, קטב לשכתי ישודDevanagari, the script used to write Hindi, Sanskrit, and many other South Asian languages, expresses certain vowels as combining marks attached to consonant letters. For example, “ह” + “​ि” = “हि” (“h” + “i” = “hi”).Korean characters stand for syllables, but they are composed of letters called jamo that stand for the vowels and consonants in the syllable. While there are code points for precomposed Korean syllables, it’s also possible to dynamically compose them by concatenating their jamo. For example, “ᄒ” + “ᅡ” + “ᆫ” = “한” (“h” + “a” + “n” = “han”).Canonical EquivalenceIn Unicode, precomposed characters exist alongside the dynamic composition system. A consequence of this is that there are multiple ways to express “the same” string—different sequences of code points that result in the same user-perceived characters. For example, as we saw earlier, we can express the character “Á” either as the single code point U+00C1, or as the string of two code points U+0041 U+0301.Another source of ambiguity is the ordering of multiple diacritics in a single character. Diacritic order matters visually when two diacritics apply to the same side of the base character, e.g. both above: “ǡ” (dot, then macron) is different from “ā̇” (macron, then dot). However, when diacritics apply to different sides of the character, e.g. one above and one below, then the order doesn’t affect rendering. Moreover, a character with multiple diacritics might have one of the diacritics precomposed and others expressed as combining marks.For example, the Vietnamese letter “ệ” can be expressed in five different ways:Fully precomposed: U+1EC7 “ệ”Partially precomposed: U+1EB9 “ẹ” + U+0302 “◌̂”Partially precomposed: U+00EA “ê” + U+0323 “◌̣”Fully decomposed: U+0065 “e” + U+0323 “◌̣” + U+0302 “◌̂”Fully decomposed: U+0065 “e” + U+0302 “◌̂” + U+0323 “◌̣”Unicode refers to set of strings like this as “canonically equivalent”. Canonically equivalent strings are supposed to be treated as identical for purposes of searching, sorting, rendering, text selection, and so on. This has implications for how you implement operations on text. For example, if an app has a “find in file” operation and the user searches for “ệ”, it should, by default, find occurrences of any of the five versions of “ệ” above!Normalization FormsTo address the problem of “how to handle canonically equivalent strings”, Unicode defines several normalization forms: ways of converting strings into a canonical form so that they can be compared code-point-by-code-point (or byte-by-byte).The “NFD” normalization form fully decomposes every character down to its component base and combining marks, taking apart any precomposed code points in the string. It also sorts the combining marks in each character according to their rendered position, so e.g. diacritics that go below the character come before the ones that go above the character. (It doesn’t reorder diacritics in the same rendered position, since their order matters visually, as previously mentioned.)The “NFC” form, conversely, puts things back together into precomposed code points as much as possible. If an unusual combination of diacritics is called for, there may not be any precomposed code point for it, in which case NFC still precomposes what it can and leaves any remaining combining marks in place (again ordered by rendered position, as in NFD).There are also forms called NFKD and NFKC. The “K” here refers to compatibility decompositions, which cover characters that are “similar” in some sense but not visually identical. However, I’m not going to cover that here.Grapheme ClustersAs we’ve seen, Unicode contains various cases where a thing that a user thinks of as a single “character” might actually be made up of multiple code points under the hood. Unicode formalizes this using the notion of a grapheme cluster: a string of one or more code points that constitute a single “user-perceived character”.UAX #29 defines the rules for what, precisely, qualifies as a grapheme cluster. It’s approximately “a base code point followed by any number of combining marks”, but the actual definition is a bit more complicated; it accounts for things like Korean jamo, and emoji ZWJ sequences.The main thing grapheme clusters are used for is text editing: they’re often the most sensible unit for cursor placement and text selection boundaries. Using grapheme clusters for these purposes ensures that you can’t accidentally chop off some diacritics when you copy-and-paste text, that left/right arrow keys always move the cursor by one visible character, and so on.Another place where grapheme clusters are useful is in enforcing a string length limit—say, on a database field. While the true, underlying limit might be something like the byte length of the string in UTF-8, you wouldn’t want to enforce that by just truncating bytes. At a minimum, you’d want to “round down” to the nearest code point boundary; but even better, round down to the nearest grapheme cluster boundary. Otherwise, you might be corrupting the last character by cutting off a diacritic, or interrupting a jamo sequence or ZWJ sequence.And More…There’s much more that could be said about Unicode from a programmer’s perspective! I haven’t gotten into such fun topics as case mapping, collation, compatibility decompositions and confusables, Unicode-aware regexes, or bidirectional text. Nor have I said anything yet about implementation issues—how to efficiently store and look-up data about the sparsely-assigned code points, or how to optimize UTF-8 decoding, string comparison, or NFC normalization. Perhaps I’ll return to some of those things in future posts.Unicode is a fascinating and complex system. It has a many-to-one mapping between bytes and code points, and on top of that a many-to-one (or, under some circumstances, many-to-many) mapping between code points and “characters”. It has oddball special cases in every corner. But no one ever claimed that representing all written languages was going to be easy, and it’s clear that we’re never going back to the bad old days of a patchwork of incompatible encodings.Further reading:The Unicode StandardUTF-8 Everywhere ManifestoDark corners of Unicode by EeveeICU (International Components for Unicode)—C/C++/Java libraries implementing many Unicode algorithms and related thingsPython 3 Unicode HowtoGoogle Noto Fonts—set of fonts intended to cover all assigned code pointsThe Many Meanings of “Shader”"

utf8 = list(map(int, text.encode("utf-8")))

In [11]:
from os import replace
# Hyper Parameter
vocab_size = 276
# This makes the numebr of replacements to be 20
num_replacement = vocab_size - 256
# copy the original to list
copy = list(utf8)

replacements = {} # recording each of the replacements (int, int) -> int

for i in range(num_replacement):
  common = find_most_common(copy)
  pair = max(common, key=common.get)
  replacement_char = 256 + i
  copy = replace_key(copy, pair, replacement_char)
  replacements[pair] = replacement_char

In [12]:
# what is the compression ratio?
len(utf8) / len(copy)

1.2656832298136647

Note that the tokenizer is a seperate object than the llm, and it has a potentially different training set, and the llm training is after this since llm only needs to read token sequences

Roughly speaking, the more training set a certain language takes up in tokenization period will lead to it being performed better

### Decoding
Token sequence to raw text

In [13]:
vocab = {char: bytes([char]) for char in range(0, 256)}
for (p0,p1), replaced in replacements.items():
  vocab[replaced] = vocab[p0] + vocab[p1]
  # bytes objects, so basically just concatenates

def decode_try(intList):
  tokens = b"".join(vocab[char] for char in intList)
  text = tokens.decode("utf-8")
  return text
# However, this may not work since the utf sometimes may need two bytes to represent an int

def decode(intList):
  tokens = b"".join(vocab[char] for char in intList)
  text = tokens.decode("utf-8", errors="replace")
  return text

decode([128])

'�'

### Encoding
Raw text to token sequence

In [14]:
def encode(text):
  # given a string, return list of integers (the tokens)
  tokens = list(text.encode("utf-8"))
  while len(tokens) >= 2:
    common = find_most_common(tokens)
    pair = min(common, key=lambda p: replacements.get(p, float("inf")))
    if pair not in replacements:
      break # nothing else can be merged
    replacement_char = replacements[pair]
    tokens = replace_key(tokens, pair, replacement_char)
  return tokens

print(encode("hello where are you"))

[104, 101, 108, 108, 111, 32, 119, 104, 260, 256, 268, 256, 121, 111, 117]


### Forced splits using regex patterns (GPT series)

In [15]:
import regex as re
gpt2pat = re.compile(r"""'s|'t|'re|'ve|'m|'ll|'d| ?\p{L}+| ?\p{N}+| ?[^\s\p{L}\p{N}]+|\s+(?!\S)|\s+""")

print(re.findall(gpt2pat, "Hello've world123 how's are you!!!?")) # find all the occurrences and split them

# Each token is individually handled and the tokenized result is concatenated

['Hello', "'ve", ' world', '123', ' how', "'s", ' are', ' you', '!!!?']


Tiktoken is the OpenAI tokenizer

In [16]:
pip install tiktoken

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 9.3 MB/s eta 0:00:00


In [17]:
import tiktoken

# GPT-2 (does not merge spaces)
enc = tiktoken.get_encoding("gpt2")
print(enc.encode("    hello world!!!"))

# GPT-4 (merges spaces)
enc = tiktoken.get_encoding("cl100k_base")
print(enc.encode("    hello world!!!"))

[220, 220, 220, 23748, 995, 10185]
[262, 24748, 1917, 12340]


Check out the GPT 2 Encoder

In [18]:
!wget https://openaipublic.blob.core.windows.net/gpt-2/models/1558M/vocab.bpe
!wget https://openaipublic.blob.core.windows.net/gpt-2/models/1558M/encoder.json

--2024-05-08 16:52:00--  https://openaipublic.blob.core.windows.net/gpt-2/models/1558M/vocab.bpe
Resolving openaipublic.blob.core.windows.net (openaipublic.blob.core.windows.net)... 20.60.179.33
Connecting to openaipublic.blob.core.windows.net (openaipublic.blob.core.windows.net)|20.60.179.33|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 456318 (446K) [application/octet-stream]
Saving to: ‘vocab.bpe’

vocab.bpe           100%[===================>] 445.62K   408KB/s    in 1.1s    

2024-05-08 16:52:02 (408 KB/s) - ‘vocab.bpe’ saved [456318/456318]

--2024-05-08 16:52:02--  https://openaipublic.blob.core.windows.net/gpt-2/models/1558M/encoder.json
Resolving openaipublic.blob.core.windows.net (openaipublic.blob.core.windows.net)... 20.60.179.33
Connecting to openaipublic.blob.core.windows.net (openaipublic.blob.core.windows.net)|20.60.179.33|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1042301 (1018K) [application/json]
Saving to: ‘

In [19]:
import os, json

with open('encoder.json', 'r') as f:
    encoder = json.load(f) # <--- ~equivalent to our "vocab" in decoding

with open('vocab.bpe', 'r', encoding="utf-8") as f:
    bpe_data = f.read()
bpe_merges = [tuple(merge_str.split()) for merge_str in bpe_data.split('\n')[1:-1]]
# ^---- ~equivalent to our "replacements"

The **replacements**, which is the dictionary of the replacements and the **vocab**, which is the byte encode concatenation of the text, and these can define an encoder

#### Special tokens

In [20]:
len(encoder) # 256 raw byte tokens. 50,000 merges. +1 special token

50257

In [21]:
encoder['<|endoftext|>'] # the only special token in use for the GPT-2 base model

50256

The special token in tiktoken does not go through the **BPE Algorithm**, instead, it has special instructions on them

These are very important for the fine tuning model, for example <|im_start|>, <|im_end|>

See GPT4's special tokens "FIM" in this ["fill in moddle" paper](https://arxiv.org/abs/2207.14255)

### sentencepiece
a commonly used library because it can both train and inference BPE tokenizers, used by llama and mistral series

Difference between **sentencepiece** and **tiktoken** is their order of the operations are different:
- Tiktoken we take the code points in the string and encode them in UTF-8 to bytes and then merge bytes
- Sentencepiece: works directly on the code points themselves, which means to merge the code points in the training set and then BPEs the code points and optionally falls back to utf-8 bytes for rare code points (rarity is determined by `character_coverage` hyperparameter), which then get translated to byte tokens.

In [23]:
import sentencepiece as spm

# write a toy.txt file with some random text
with open("toy.txt", "w", encoding="utf-8") as f:
  f.write("SentencePiece is an unsupervised text tokenizer and detokenizer mainly for Neural Network-based text generation systems where the vocabulary size is predetermined prior to the neural model training. SentencePiece implements subword units (e.g., byte-pair-encoding (BPE) [Sennrich et al.]) and unigram language model [Kudo.]) with the extension of direct training from raw sentences. SentencePiece allows us to make a purely end-to-end system that does not depend on language-specific pre/postprocessing.")

#### Llama2 tokenizer using sentencepiece

In [24]:
import os

options = dict(
  # input: raw text
  input="toy.txt",
  input_format="text",
  # output file name prefix
  model_prefix="tok400",
  # algorithm spec
  # BPE alg
  model_type="bpe",
  vocab_size=400,
  # normalization and preprocessing
  normalization_rule_name="identity", # turn off normalization for most llm cases, keep the data as raw form
  remove_extra_whitespaces=False,
  input_sentence_size=200000000, # max number of training sentences
  max_sentence_length=4192, # max number of bytes per sentence
  seed_sentencepiece_size=1000000,
  shuffle_input_sentence=True,
  # rare word treatment
  character_coverage=0.99995,
  byte_fallback=True,
  # merge rules
  split_digits=True,
  split_by_unicode_script=True,
  split_by_whitespace=True,
  split_by_number=True,
  max_sentencepiece_length=16,
  add_dummy_prefix=True,
  allow_whitespace_only_pieces=True,
  # special tokens
  unk_id=0, # the UNK token MUST exist
  bos_id=1, # the others are optional, set to -1 to turn off
  eos_id=2,
  pad_id=-1,
  # systems
  num_threads=os.cpu_count(), # use ~all system resources
)

spm.SentencePieceTrainer.train(**options)

In [25]:
sp = spm.SentencePieceProcessor()
sp.load('tok400.model')
vocab = [[sp.id_to_piece(idx), idx] for idx in range(sp.get_piece_size())]
vocab

[['<unk>', 0],
 ['<s>', 1],
 ['</s>', 2],
 ['<0x00>', 3],
 ['<0x01>', 4],
 ['<0x02>', 5],
 ['<0x03>', 6],
 ['<0x04>', 7],
 ['<0x05>', 8],
 ['<0x06>', 9],
 ['<0x07>', 10],
 ['<0x08>', 11],
 ['<0x09>', 12],
 ['<0x0A>', 13],
 ['<0x0B>', 14],
 ['<0x0C>', 15],
 ['<0x0D>', 16],
 ['<0x0E>', 17],
 ['<0x0F>', 18],
 ['<0x10>', 19],
 ['<0x11>', 20],
 ['<0x12>', 21],
 ['<0x13>', 22],
 ['<0x14>', 23],
 ['<0x15>', 24],
 ['<0x16>', 25],
 ['<0x17>', 26],
 ['<0x18>', 27],
 ['<0x19>', 28],
 ['<0x1A>', 29],
 ['<0x1B>', 30],
 ['<0x1C>', 31],
 ['<0x1D>', 32],
 ['<0x1E>', 33],
 ['<0x1F>', 34],
 ['<0x20>', 35],
 ['<0x21>', 36],
 ['<0x22>', 37],
 ['<0x23>', 38],
 ['<0x24>', 39],
 ['<0x25>', 40],
 ['<0x26>', 41],
 ['<0x27>', 42],
 ['<0x28>', 43],
 ['<0x29>', 44],
 ['<0x2A>', 45],
 ['<0x2B>', 46],
 ['<0x2C>', 47],
 ['<0x2D>', 48],
 ['<0x2E>', 49],
 ['<0x2F>', 50],
 ['<0x30>', 51],
 ['<0x31>', 52],
 ['<0x32>', 53],
 ['<0x33>', 54],
 ['<0x34>', 55],
 ['<0x35>', 56],
 ['<0x36>', 57],
 ['<0x37>', 58],
 ['<0x38>', 5

In [26]:
ids = sp.encode("hello 你好么")
print(ids)

[362, 378, 361, 372, 358, 362, 231, 192, 163, 232, 168, 192, 231, 188, 139]


These Chinese characters are not part of the training set, but because we have set `byte_fallback` to `True`, these characters fall back to bytes,

In [27]:
print([sp.id_to_piece(idx) for idx in ids])

['▁', 'h', 'e', 'l', 'lo', '▁', '<0xE4>', '<0xBD>', '<0xA0>', '<0xE5>', '<0xA5>', '<0xBD>', '<0xE4>', '<0xB9>', '<0x88>']


Why do we have an extra space in the front? This is because we have set `add_dummy_prefix` to be `True`. We need this since in tiktokenizer words in the front of the sentence are different encoding then the same word inside the sentence, and the llm will need to learn that they are very similar in context.

What will happen if we set `byte_fallback` to `False`? Then, any unknown characters will be 0, since unk = 0. Then "你好么" will be 0, and decoded as unk

#### Vocab Size

- Q: what should be vocab size?
- Q: how can I increase vocab size?
- A: let's see. Reminder: [gpt.py](https://github.com/karpathy/ng-video-lecture/blob/master/gpt.py) from before.